In [3]:
import pandas as pd
import numpy as np

In [4]:
# ============================================
# PART 1 — Setup + Load Model
# ============================================

import os
import torch
import pickle
from tqdm import tqdm
from PIL import Image
from sentence_transformers import SentenceTransformer, util
from google.colab import files

# Load CLIP model (cached)
print("🚀 Loading CLIP model...")
model = SentenceTransformer('clip-ViT-B-32', cache_folder="/content/clip_cache")

# Dataset folder & embedding file
dataset_folder = "/content/drive/MyDrive/Datasets/polyvore-outfit-dataset/polyvore_outfits/images"
embedding_file = "/content/fashion_embeddings.pkl"


ModuleNotFoundError: No module named 'torch'

In [ ]:
# ============================================
# 📦 Polyvore Outfit Dataset Setup (Full Cell)
# ============================================

import os
from google.colab import drive

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Define paths
zip_path = "/content/drive/MyDrive/polyvore-outfit-dataset.zip"   # <-- your zip file
extract_dir = "/content/drive/MyDrive/Datasets/polyvore-outfit-dataset"
images_dir = os.path.join(extract_dir, "polyvore_outfits", "images")

# 3️⃣ Check if already extracted
if os.path.exists(images_dir):
    print("✅ Dataset already extracted at:", images_dir)
else:
    # Create folder if not exists
    os.makedirs("/content/drive/MyDrive/Datasets", exist_ok=True)

    # 4️⃣ Extract from ZIP
    print("📦 Extracting dataset... Please wait (may take a few minutes)...")
    !unzip -q "{zip_path}" -d "/content/drive/MyDrive/Datasets/"
    print("✅ Extraction complete!")

# 5️⃣ Verify images folder
if os.path.exists(images_dir):
    print("✅ Images found at:", images_dir)
    print("📸 Sample files:", len(os.listdir(images_dir)), "images found.")
else:
    print("❌ Images folder not found! Please check zip structure.")


In [ ]:
# ============================================
# PART 2 — Compute and Save Embeddings (Run Once)
# ============================================

dataset_folder = "/content/drive/MyDrive/Datasets/polyvore-outfit-dataset/polyvore_outfits/images"

if not os.path.exists(embedding_file):
    print("⚙️ Computing image embeddings... (one-time process)")
    images = [os.path.join(dataset_folder, img) for img in os.listdir(dataset_folder)[:1000]]
    image_embeddings = []
    for img_path in tqdm(images):
        try:
            img = Image.open(img_path).convert("RGB")
            emb = model.encode(img, convert_to_tensor=True, show_progress_bar=False)
            image_embeddings.append((img_path, emb))
        except:
            pass
    with open(embedding_file, "wb") as f:
        pickle.dump(image_embeddings, f)
    print("✅ Embeddings saved successfully!")
else:
    print("✅ Embeddings already exist. Skip this part next time.")

⚙️ Computing image embeddings... (one-time process)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Datasets/polyvore-outfit-dataset/polyvore_outfits/images'

In [ ]:
# ============================================
# 🧠 Step 2: Create or Load Embeddings
# ============================================
import os
from tqdm import tqdm
from PIL import Image
import torch
from sentence_transformers import SentenceTransformer, util
import pickle

# Model (you can also use "clip-ViT-B-32" for speed)
model = SentenceTransformer("clip-ViT-B-32")

# Dataset path (same as before)
dataset_folder = "/content/drive/MyDrive/Datasets/polyvore-outfit-dataset/polyvore_outfits/images"

# File to save embeddings
embed_file = "/content/drive/MyDrive/Datasets/polyvore_embeddings.pkl"

# If embeddings already exist, load them
if os.path.exists(embed_file):
    print("✅ Loading precomputed embeddings...")
    with open(embed_file, "rb") as f:
        image_embeddings = pickle.load(f)
else:
    print("⚙️ Creating embeddings (this may take 10–15 mins for all images)...")
    image_embeddings = []
    image_files = [os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder) if f.endswith(".jpg")]

    for img_path in tqdm(image_files[:3000]):  # limit to first 3k to stay under 3GB
        try:
            img = Image.open(img_path).convert("RGB")
            emb = model.encode(img, convert_to_tensor=True)
            image_embeddings.append((img_path, emb))
        except Exception as e:
            print("⚠️ Error:", img_path, e)

    # Save embeddings
    with open(embed_file, "wb") as f:
        pickle.dump(image_embeddings, f)
    print(f"✅ Saved embeddings to {embed_file}")

print(f"📦 Total embedded images: {len(image_embeddings)}")


In [ ]:
# ============================================
# 👗 Step 3: Smart Outfit Recommender (Enhanced)
# ============================================
import os
from PIL import Image
import torch
from sentence_transformers import SentenceTransformer, util
import pickle
from google.colab import files
from IPython.display import display, Image as IPyImage

# Load model + embeddings
model = SentenceTransformer("clip-ViT-B-32")
embed_file = "/content/drive/MyDrive/Datasets/polyvore_embeddings.pkl"

with open(embed_file, "rb") as f:
    image_embeddings = pickle.load(f)

# -----------------------------------------
# 🧠 Upload image from local system
# -----------------------------------------
print("📤 Please upload your photo...")
uploaded = files.upload()   # opens file picker
user_image_path = list(uploaded.keys())[0]
print("✅ Uploaded:", user_image_path)

# -----------------------------------------
# 🧠 Text + Gender input
# -----------------------------------------
user_text = input("📝 Describe your occasion (e.g., I am going to a party): ")
gender = input("👤 Enter gender (Male/Female): ").strip().lower()

# -----------------------------------------
# 🔍 Encode User Inputs
# -----------------------------------------
user_img = Image.open(user_image_path).convert("RGB")
user_img_emb = model.encode(user_img, convert_to_tensor=True)
text_emb = model.encode(user_text, convert_to_tensor=True)

# -----------------------------------------
# 💬 Suitability Analysis
# -----------------------------------------
fit_score = util.cos_sim(user_img_emb, text_emb).item()

print("\n🧠 Suitability Analysis:")
if fit_score > 0.45:
    print(f"✅ Your outfit seems suitable for this occasion! (score: {fit_score:.2f})")
else:
    print(f"❌ Your outfit may not fit well for this occasion. (score: {fit_score:.2f})")

# -----------------------------------------
# 💡 Recommendation Logic (with gender filter)
# -----------------------------------------
scores = []
for img_path, emb in image_embeddings:
    # Optional gender-based filter (if filenames contain hints)
    if gender == "male" and "women" in img_path.lower():
        continue
    if gender == "female" and "men" in img_path.lower():
        continue

    sim = 0.6 * util.cos_sim(text_emb, emb).item() + 0.4 * util.cos_sim(user_img_emb, emb).item()
    scores.append((img_path, sim))

# Top 5 results
top_results = sorted(scores, key=lambda x: x[1], reverse=True)[:5]

# -----------------------------------------
# 🖼️ Display Recommendations
# -----------------------------------------
print("\n✨ Recommended Outfits 👕\n")
for path, score in top_results:
    print(f"Score: {score:.4f} | {os.path.basename(path)}")
    display(IPyImage(filename=path))
